## Step 1: Environment Setup

First, we'll check GPU availability and install required packages.

In [ ]:
# Check GPU availability
!nvidia-smi

print("\n" + "="*70)
print("GPU STATUS")
print("="*70)
print("If you see GPU information above, GPU acceleration is enabled!")
print("If not, go to Runtime > Change runtime type > GPU")
print("="*70)

In [ ]:
# Install required packages
print("Installing dependencies... (this takes 2-3 minutes)\n")

!pip install -q qiskit
!pip install -q numpy scipy pandas networkx matplotlib seaborn

# Try to install GPU support (may not work on all Colab instances)
!pip install -q qiskit-aer-gpu-cu11 2>/dev/null || echo "GPU support not available—please enable a GPU runtime"

print("\n✓ Installation complete!")

In [ ]:
# Verify GPU-enabled Qiskit installation
import importlib.metadata as metadata
import qiskit
import qiskit_aer
from qiskit_aer.backends.aer_simulator import AerSimulator

print(f"Qiskit version: {qiskit.__version__}")
print(f"Qiskit Aer version: {qiskit_aer.__version__}")

try:
    gpu_pkg_version = metadata.version('qiskit-aer-gpu-cu11')
    print(f"qiskit-aer-gpu-cu11 version: {gpu_pkg_version}")
except metadata.PackageNotFoundError:
    print("⚠️ qiskit-aer-gpu-cu11 package not found")

try:
    AerSimulator(method='statevector', device='GPU')
    print("\n✓ GPU Aer simulator available")
except Exception as exc:
    print(f"\n⚠️ GPU Aer simulator unavailable: {exc}")

## Step 2: Get Source Code

You have two options to get the source code:

**Option A (Recommended)**: Clone from GitHub repository
- Faster and easier
- Always gets the latest version
- Run the cell below

**Option B**: Upload files manually
- Use if you have local modifications
- Upload via file browser on the left
- Need these 7 files: `network_generation.py`, `noise_models.py`, `qaoa_optimizer.py`, `classical_baselines.py`, `run_experiments.py`, `visualize_results.py`, `demo.py`

In [ ]:
# Option A: Clone from GitHub (RECOMMENDED)
print("Cloning repository from GitHub...\n")

!git clone https://github.com/atharvk542/noise_aware_qaoa.git

# Move files to current directory
!cp noise_aware_qaoa/*.py .

print("\n✓ Repository cloned successfully!")

# Verify all required files are present
required_files = [
    'network_generation.py',
    'noise_models.py', 
    'qaoa_optimizer.py',
    'classical_baselines.py',
    'run_experiments.py',
    'visualize_results.py',
    'demo.py'
]

import os
missing = [f for f in required_files if not os.path.exists(f)]

if missing:
    print("\n⚠️ Missing files:")
    for f in missing:
        print(f"  - {f}")
else:
    print("\n✓ All required files present!")
    print("\nFiles ready:")
    for f in required_files:
        print(f"  - {f}")

In [ ]:
# Option B: Manual file upload (if you have local modifications)
# Uncomment and run this cell instead of Option A if needed

# from google.colab import files
# 
# print("Please upload the 7 Python files from your workspace...\n")
# uploaded = files.upload()
# 
# print("\n✓ Files uploaded:")
# for filename in uploaded.keys():
#     print(f"  - {filename}")
# 
# # Verify all required files are present
# required_files = [
#     'network_generation.py',
#     'noise_models.py', 
#     'qaoa_optimizer.py',
#     'classical_baselines.py',
#     'run_experiments.py',
#     'visualize_results.py',
#     'demo.py'
# ]
# 
# import os
# missing = [f for f in required_files if not os.path.exists(f)]
# 
# if missing:
#     print("\n⚠️ Missing files:")
#     for f in missing:
#         print(f"  - {f}")
# else:
#     print("\n✓ All required files present!")

## Step 3: Quick Demo Test

Run a quick demo to verify everything works (takes 2-3 minutes).

In [ ]:
# this takes a long time don't do this one loool
!python demo.py

## Step 4: Run Optimized Experiments

This runs the publication-quality experimental suite:
- 16 configurations (2 networks × 2 sizes × 2 demands × 2 error rates)
- 10 trials each for statistical validity
- Total: 160 QAOA optimizations

**Expected runtime**: 
- With GPU: 8-12 hours
- Without GPU: 24-36 hours (may exceed Colab's 12-hour limit)

**Note**: Colab sessions disconnect after 12 hours. Make sure to download results periodically!

In [ ]:
# Create results directory
!mkdir -p results_colab
!mkdir -p figures_colab

print("✓ Results directories created")

In [ ]:
import time

print("="*70)
print("STARTING OPTIMIZED EXPERIMENTAL SUITE")
print("="*70)
print(f"Start time: {time.strftime('%Y-%m-%d %H:%M:%S')}")
print("\nConfiguration:")
print("  - Network types: barbell, grid")
print("  - Node counts: 8, 10")
print("  - Demand counts: 3, 4")
print("  - Gate errors: 0.001, 0.01")
print("  - Trials per config: 10")
print("  - Total QAOA runs: 160")
print("\nThis will take 8-12 hours with GPU. Progress updates every 30 minutes.")
print("="*70 + "\n")

start_time = time.time()

!python run_experiments.py \
    --network-types barbell grid \
    --num-nodes 8 10 \
    --num-demands 3 4 \
    --gate-errors 0.001 0.01 \
    --num-trials 10 \
    --output-dir results_colab

end_time = time.time()
elapsed_hours = (end_time - start_time) / 3600

print("\n" + "="*70)
print("EXPERIMENTS COMPLETE!")
print("="*70)
print(f"Total runtime: {elapsed_hours:.2f} hours")
print(f"End time: {time.strftime('%Y-%m-%d %H:%M:%S')}")
print("="*70)

### Checkpoint: Download Intermediate Results

Run this periodically to save your progress in case Colab disconnects.

In [ ]:
from google.colab import files
import os

if os.path.exists('results_colab/experiment_results.csv'):
    files.download('results_colab/experiment_results.csv')
    print("✓ Downloaded experiment_results.csv")
    
if os.path.exists('results_colab/significance_tests.csv'):
    files.download('results_colab/significance_tests.csv')
    print("✓ Downloaded significance_tests.csv")
else:
    print("⚠️ No results found yet. Experiments may still be running.")

## Step 5: Generate Visualizations

Create publication-quality figures from your results.

In [ ]:
print("Generating publication-quality figures...\n")

!python visualize_results.py \
    results_colab/experiment_results.csv \
    --significance-file results_colab/significance_tests.csv \
    --output-dir figures_colab

print("\n✓ Figures generated in figures_colab/")

### Preview Generated Figures

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

# List all generated figures
figure_dir = 'figures_colab'
figures = [f for f in os.listdir(figure_dir) if f.endswith('.png')]

print(f"Generated {len(figures)} figures:\n")
for fig in figures:
    print(f"  - {fig}")

# Display each figure
for fig_name in figures:
    fig_path = os.path.join(figure_dir, fig_name)
    img = mpimg.imread(fig_path)
    
    plt.figure(figsize=(12, 8))
    plt.imshow(img)
    plt.axis('off')
    plt.title(fig_name, fontsize=14, pad=20)
    plt.tight_layout()
    plt.show()

## Step 6: Analyze Results

Quick summary of experimental findings.

In [ ]:
import pandas as pd
import numpy as np

# Load results
results_df = pd.read_csv('results_colab/experiment_results.csv')
sig_df = pd.read_csv('results_colab/significance_tests.csv')

print("="*70)
print("EXPERIMENTAL RESULTS SUMMARY")
print("="*70)

print(f"\nTotal experiments: {len(results_df)}")
print(f"Configurations: {len(sig_df)}")

# Overall statistics
print("\n" + "-"*70)
print("OVERALL PERFORMANCE")
print("-"*70)
print(f"QAOA utility:   {results_df['qaoa_utility'].mean():.4f} ± {results_df['qaoa_utility'].std():.4f}")
print(f"Greedy utility: {results_df['greedy_utility'].mean():.4f} ± {results_df['greedy_utility'].std():.4f}")
print(f"QAOA advantage: {results_df['qaoa_advantage'].mean()*100:.2f}% ± {results_df['qaoa_advantage'].std()*100:.2f}%")

# Statistical significance
significant = sig_df[sig_df['p_value'] < 0.05]
print("\n" + "-"*70)
print("STATISTICAL SIGNIFICANCE")
print("-"*70)
print(f"Configurations where QAOA significantly better: {len(significant)}/{len(sig_df)}")
print(f"Average p-value: {sig_df['p_value'].mean():.4f}")
print(f"Average Cohen's d: {sig_df['cohens_d'].mean():.3f}")

# Best configuration
best_config = sig_df.loc[sig_df['mean_advantage'].idxmax()]
print("\n" + "-"*70)
print("BEST QAOA ADVANTAGE")
print("-"*70)
print(f"Network: {best_config['network_type']}")
print(f"Nodes: {int(best_config['num_nodes'])}")
print(f"Demands: {int(best_config['num_demands'])}")
print(f"Gate error: {best_config['gate_error_2q']:.4f}")
print(f"Advantage: {best_config['mean_advantage']:.4f}")
print(f"p-value: {best_config['p_value']:.4f}")
print(f"Cohen's d: {best_config['cohens_d']:.3f}")

print("\n" + "="*70)
print("KEY FINDINGS")
print("="*70)

# Find trends
high_contention = results_df[results_df['contention_level'] > results_df['contention_level'].median()]
low_contention = results_df[results_df['contention_level'] <= results_df['contention_level'].median()]

print(f"\n1. Contention Level Effect:")
print(f"   High contention QAOA advantage: {high_contention['qaoa_advantage'].mean()*100:.2f}%")
print(f"   Low contention QAOA advantage: {low_contention['qaoa_advantage'].mean()*100:.2f}%")

high_noise = results_df[results_df['gate_error_2q'] > results_df['gate_error_2q'].median()]
low_noise = results_df[results_df['gate_error_2q'] <= results_df['gate_error_2q'].median()]

print(f"\n2. Noise Resilience:")
print(f"   Low noise (0.1%) QAOA utility: {low_noise['qaoa_utility'].mean():.4f}")
print(f"   High noise (1%) QAOA utility: {high_noise['qaoa_utility'].mean():.4f}")
print(f"   Degradation: {(1 - high_noise['qaoa_utility'].mean()/low_noise['qaoa_utility'].mean())*100:.1f}%")

print(f"\n3. Approximation Quality:")
print(f"   QAOA approximation ratio: {results_df['qaoa_approximation_ratio'].mean():.4f}")
print(f"   Greedy approximation ratio: {results_df['greedy_approximation_ratio'].mean():.4f}")
print(f"   QAOA closer to optimal by: {(results_df['qaoa_approximation_ratio'].mean() - results_df['greedy_approximation_ratio'].mean())*100:.1f}%")

print("\n" + "="*70)

## Step 7: Download Results

Download all results and figures to your local machine.

In [ ]:
from google.colab import files
import os

print("Downloading results and figures...\n")

# Download CSV results
csv_files = [
    'results_colab/experiment_results.csv',
    'results_colab/significance_tests.csv'
]

for csv_file in csv_files:
    if os.path.exists(csv_file):
        files.download(csv_file)
        print(f"✓ Downloaded {csv_file}")

# Download all figures
figure_dir = 'figures_colab'
if os.path.exists(figure_dir):
    for fig_file in os.listdir(figure_dir):
        if fig_file.endswith('.png'):
            fig_path = os.path.join(figure_dir, fig_file)
            files.download(fig_path)
            print(f"✓ Downloaded {fig_path}")

print("\n✓ All downloads complete!")
print("\nYou can now use these files in your publication.")

## Optional: Create ZIP Archive

Download all results as a single ZIP file.

In [ ]:
!zip -r colab_results.zip results_colab/ figures_colab/

from google.colab import files
files.download('colab_results.zip')

print("✓ Downloaded colab_results.zip")

---

## Optional: Custom Experiments

Run custom configurations if you need different parameters.

In [ ]:
# Example: Test only barbell networks with more trials
!python run_experiments.py \
    --network-types barbell \
    --num-nodes 8 10 12 \
    --num-demands 3 4 5 \
    --gate-errors 0.001 0.01 \
    --num-trials 15 \
    --output-dir results_custom

In [ ]:
# Visualize custom results
!python visualize_results.py \
    results_custom/experiment_results.csv \
    --significance-file results_custom/significance_tests.csv \
    --output-dir figures_custom

---

## Tips for Using This Notebook

### 1. **Keep Session Alive**
- Colab disconnects after 12 hours or 90 minutes of inactivity
- Use browser extensions like "Colab Alive" to prevent disconnection
- Run checkpoint downloads periodically

### 2. **Monitor Progress**
- Check the runtime logs for progress updates
- Each configuration prints its status
- Look for "Experiment X/16 complete" messages

### 3. **GPU Acceleration**
- Verify GPU is enabled: Runtime > Change runtime type > GPU
- Check `nvidia-smi` output in first cell
- GPU should show ~5-10× speedup vs CPU

### 4. **If Session Disconnects**
- Re-upload your Python files
- Results are saved in `results_colab/` (lost on disconnect)
- Download checkpoints regularly to prevent data loss

### 5. **Reduce Runtime**
- Decrease `--num-trials` (10 → 5 for quick test)
- Test fewer configurations first
- Use `demo.py` to verify setup before long runs

### 6. **For Paper Submission**
- Download both CSV files and all PNG figures
- CSV files contain raw data for tables
- PNG figures are publication-ready (300 DPI)
- Statistical tests in `significance_tests.csv`

---

## Citation

If you use this code in your research, please cite:

```bibtex
@article{noise_robust_qaoa_routing_2025,
  title={Noise-Robust QAOA for Multi-Flow Entanglement Routing in Quantum Repeater Networks},
  author={[Your Name]},
  journal={[Conference/Journal]},
  year={2025}
}
```

---

## Support

For issues or questions:
1. Check `README.md` in the GitHub repository
2. Review `PERFORMANCE_GUIDE.md` for optimization details
3. Consult `QUICKSTART.md` for parameter explanations

**Expected completion**: Upload files → Run experiments → Download results takes 8-12 hours total with GPU.

**Good luck with your research!** 🚀